# Constants

Values computed in [constants.ipynb](./constants.ipynb)

In [55]:
TRANSPORT_DAY_PERSON = 6.583113447812
OFFICE_DAY_PERSON = 5.495107632093934
MAN_DAY_CO2 = TRANSPORT_DAY_PERSON + OFFICE_DAY_PERSON

SERVER_DAY = 2.9480188516448917
CO2_GB = 0.0015
DISK_DAY = 0.013110912000000002

USER_HOUR_CO2 = 0.026848423462431416

# Model

In [56]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
import ipywidgets.widgets as widgets

class Model:
    """
    Software lifecycle carbon cost model class
    """
    def __init__(self):
        #Inputs
        self.inputs = {
            "disks": ModelInput(1000, 3000, 'Number of disks: '),
            "servers" : ModelInput(100, 300, 'Number of servers : '),
            "data_transferred" : ModelInput(1000000, 30000000, 'Data transferred: '),
            "user_hours" : ModelInput(150000, 300000, 'Users hours:'),
            "spec" : ModelInput(200, 300, 'Specifications man-days:'),
            "dev" : ModelInput(2000, 3000, 'Dev man-days:'),
            "design" : ModelInput(200, 3000, 'Design man-days:'),
            "test" : ModelInput(200, 3000, 'Test man-days:'),
            "dev_servers" : ModelInput(4, 30, 'Number of dev servers:'),
            "support" : ModelInput(50, 300, 'Support man-days:'),
            "maintenance" : ModelInput(500, 3000, 'Maintenance man-days:'),
            "run_length" : ModelInput(1000, 3000, 'Run phase duration:'),
            "build_length" : ModelInput(1500, 3000, 'Build phase duration:'),
        }
        self.project_length = None

        # Outputs
        self.network_co2 = None
        self.management_co2 = None
        self.support_co2 = None
        self.servers_co2 = None
        self.implementation_co2 = None
        self.spec_co2 = None
        self.test_co2 = None
        self.design_co2 = None
        self.dev_co2 = None
        self.run_co2 = None
        self.build_co2 = None
        self.infra_dev_co2 = None
        self.maintenance_co2 = None
        self.storage_co2 = None
        self.infra_co2 = None
        self.user_co2 = None

        self.compute()

    def compute(self):
        """
        Compute the phases carbon cost retrieving the inputs from the sliders
        """
        self.project_length = self.inputs["run_length"].get_value() - self.inputs["build_length"].get_value()
        self.compute_build()
        self.compute_run()

        
    def compute_build(self):
        """
        Compute the build phase outputs
        """

        ## Requirement analysis and specifications
        self.spec_co2 = self.inputs["spec"].get_value() * MAN_DAY_CO2

        ## Implementation
        self.infra_dev_co2 = self.inputs["dev_servers"].get_value() * SERVER_DAY * self.inputs["build_length"].get_value()
        self.dev_co2 = self.inputs["dev"].get_value() * MAN_DAY_CO2
        self.design_co2 = self.inputs["design"].get_value() * MAN_DAY_CO2
        self.test_co2 = self.inputs["test"].get_value() * MAN_DAY_CO2
        self.implementation_co2 = self.inputs["dev"].get_value() * MAN_DAY_CO2 + self.infra_dev_co2

        # Management
        self.management_co2 = self.inputs["build_length"].get_value() * MAN_DAY_CO2

        # Global
        self.build_co2 = self.spec_co2 + self.implementation_co2 + self.management_co2

    def compute_run(self):
        """
        Compute the run phase outputs
        """

        # Support
        self.support_co2 = self.inputs["support"].get_value() * MAN_DAY_CO2

        # Maintenance
        self.maintenance_co2 = self.inputs["maintenance"].get_value() * MAN_DAY_CO2

        # Infra
        self.network_co2 = self.inputs["data_transferred"].get_value() * CO2_GB
        self.servers_co2 = self.inputs["servers"].get_value() * self.inputs["run_length"].get_value() * SERVER_DAY
        self.storage_co2 = self.inputs["disks"].get_value() * self.inputs["run_length"].get_value() * DISK_DAY

        self.infra_co2 = self.network_co2 + self.servers_co2 + self.storage_co2

        # User
        self.user_co2 = self.inputs["user_hours"].get_value() * USER_HOUR_CO2

        # Global
        self.run_co2 = self.support_co2 + self.maintenance_co2 + self.infra_co2 + self.user_co2


class ModelInput:
    """
    Inputs from the model, gives sliders and corresponding values
    """
    def __init__(self, value, max_value, desc):
        style = {'description_width': 'initial'}
        self.widget = widgets.IntSlider(min=0, max=max_value, value= value, description = desc, layout=widgets.Layout(width='400px'), style = style)

    def get_value(self):
        return self.widget.value

# Output

In [57]:
def draw_model(model: Model):
    """
    Pie chart model representation
    :param model: Model to pie
    """
    fig1, ax1 = plt.subplots()
    ax1.pie([model.build_co2, model.run_co2], labels=["Build", "Run"])
    ax1.set_title("Global")

    fig, ((ax2, ax3), (ax4, ax5)) = plt.subplots(2, 2, constrained_layout=True)
    ax2.pie(
        [model.spec_co2, model.implementation_co2, model.management_co2],
        labels=["Spec", "Implementation", "Management"],
    )
    ax2.set_title("Build")
    ax3.pie(
        [model.dev_co2, model.design_co2, model.test_co2, model.infra_dev_co2],
        labels=["Dev", "Design", "Test", "Infra"],
    )
    ax3.set_title("Implementation")

    ax4.pie(
        [model.support_co2, model.maintenance_co2, model.infra_co2, model.user_co2],
        labels=["Support", "Maintenance", "Infra", "User"],
    )
    ax4.set_title("Run")
    ax5.pie(
        [model.network_co2, model.storage_co2, model.servers_co2],
        labels=["Network", "Storage", "Compute"],
    )
    ax5.set_title("Infra")
    plt.show()


model = Model()

# Button refresh
button_refresh = widgets.Button(description="Refresh")

def on_button_clicked(b):
    with out:
        display.clear_output(True)
        model.compute()
        draw_model(model)

button_refresh.on_click(on_button_clicked)

# Sliders
vbox_build = widgets.VBox(
    [
        widgets.Text(value="Build: ", disabled=True),
        model.inputs["spec"].widget,
        model.inputs["dev"].widget,
        model.inputs["design"].widget,
        model.inputs["test"].widget,
        model.inputs["dev_servers"].widget,
        model.inputs["build_length"].widget,
    ]
)

vbox_run = widgets.VBox(
    [
        widgets.Text(value="Run: ", disabled=True),
        model.inputs["disks"].widget,
        model.inputs["servers"].widget,
        model.inputs["data_transferred"].widget,
        model.inputs["user_hours"].widget,
        model.inputs["support"].widget,
        model.inputs["maintenance"].widget,
        model.inputs["run_length"].widget,
    ]
)

vbox_sliders = widgets.VBox([vbox_build, vbox_run, button_refresh])

# Graph
out = widgets.Output()
with out:
    draw_model(model)


# Global
widgets.HBox([vbox_sliders, out])